# Given example and template, demonstrate how chat adapter works

In [1]:
%load_ext autoreload
%autoreload 2

try: import google.colab; root_path = 'dsp'
except: root_path = '.'

import os
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

import dsp

openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key (optional)
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'

Not loading Cohere because it is not installed.


In [31]:
train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

# Train is an array of examples
train = [dsp.Example(question=question, answer=answer) for question, answer in train]

In [32]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']),
       ('What documentary about the Gilgo Beach Killer debuted on A&E?', ['The Killing Season']),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', ['Gainesville, Florida', 'Gainesville']),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', ['1828']),
       ('Which author is English: John Braine or Studs Terkel?', ['John Braine']),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', ['Andre Dirrell']),
       ('In which city is the sports nutrition business established by Oliver Cookson based ?', ['Cheshire', 'Cheshire, UK']),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', ['February 13, 1980']),
       ('Kyle Moran was born in the town on what river?', ['Castletown', 'Castletown River']),
       ("What is the name of one branch of Robert D. Braun's speciality?", ['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']),
       ("Where was the actress who played the niece in the Priest film born?", ['Surrey', 'Guildford, Surrey']),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', ['Portrait of a Marriage']),
       ('What year was the father of the Princes in the Tower born?', ['1442'])]

dev = [dsp.Example(question=question, answer=answer) for question, answer in dev]

Basic Adapter Input: Template with Example

Basic Adapter Output: Prompt String

Chat Adapter Input: Template with Example

Chat Adapter Output: Prompt Dictionairy

In [33]:
# Template, Demos, and Question Construction
Question = dsp.Type(prefix="Question:", desc="${the question to be answered}")
Answer = dsp.Type(prefix="Answer:", desc="${a short factoid answer, often between 1 and 5 words}", format=dsp.format_answers)
qa_template = dsp.Template(instructions="Answer questions with short factoid answers.", question=Question(), answer=Answer())

question = dev[0].question
demos = dsp.sample(train, k=1)
example = dsp.Example(question=question, demos=demos)

### Load Models and Adapters

In [34]:
rm = dsp.ColBERTv2(url=colbert_server)
davinci = dsp.GPT3(model='text-davinci-002', api_key=openai_key)
turbo = dsp.GPT3(model='gpt-3.5-turbo', api_key=openai_key, model_type="chat")


basic_adapter = dsp.modules.BasicAdapter()
chat_adapter = dsp.modules.ChatAdapter()

### Basic Adapter with demos


In [35]:
dsp.settings.configure(lm=davinci, rm=rm)

parts = qa_template(example)
prompt = basic_adapter(parts)

print("------")
print("PARTS")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("PROMPT")
print(prompt)

------
PARTS
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
PROMPT
Answer questions with short factoid answers.

---

Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer:


### Chat Adapter with demos

In [36]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Basic Adapter with no demos

In [37]:
dsp.settings.configure(lm=davinci, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = basic_adapter(parts)

print("------")
print("PARTS")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("PROMPT")
print(prompt)

------
PARTS
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
PROMPT
Answer questions with short factoid answers.

---

Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}

---

Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer:


### Chat Adapter with no demos


In [38]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts)

print("------")
print("PARTS")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("PROMPT")
print(prompt)

------
PARTS
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
PROMPT
{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}]}


### Basic Adapter with demos and context

In [39]:
Context = dsp.Type(
    prefix="Context:\n",
    desc="${sources that may contain relevant content}",
    format=dsp.passages2text
)

qa_template_with_passages = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), answer=Answer()
)

passages = dsp.retrieve(question, k=1)
example = dsp.Example(question=question, context=passages, demos=demos)

In [40]:
dsp.settings.configure(lm=davinci, rm=rm)
parts = qa_template_with_passages(example)
prompt = basic_adapter(parts)

print("------")
print("PASSAGES")
for i, v in enumerate(passages):
    print(str(i)+":")
    print(v)

print("------")
print("PARTS")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("PROMPT")
print(prompt)

------
PASSAGES
0:
Julia Peterkin | Julia Peterkin Julia Peterkin (October 31, 1880 – August 10, 1961) was an American author from South Carolina. In 1929 she won the Pulitzer Prize for Novel/Literature, for her novel "Scarlet Sister Mary." She wrote several novels about the plantation South, especially the Gullah people of the Low Country. She was one of the few white authors who wrote about the African-American experience. Julia Mood was born in Laurens County, South Carolina. Her father was a physician, and she was the third of his four children. Her mother died soon after her birth, and her father later married Janie Brogdon.
------
PARTS
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Context:
${sources that may contain relevant content}

Question: ${the question to be answered}

Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. 

### Chat Adapter with demos and context

In [41]:
dsp.settings.configure(lm=turbo, rm=rm)

parts = qa_template_with_passages(example)
prompt = chat_adapter(parts)

print("------")
print("PASSAGES")
for i, v in enumerate(passages):
    print(str(i)+":")
    print(v)

print("------")
print("PARTS")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("PROMPT")
print(prompt['model'])
print("messages")
for i, v in enumerate(prompt['messages']):
    print(str(i)+":")
    print(v)

------
PASSAGES
0:
Julia Peterkin | Julia Peterkin Julia Peterkin (October 31, 1880 – August 10, 1961) was an American author from South Carolina. In 1929 she won the Pulitzer Prize for Novel/Literature, for her novel "Scarlet Sister Mary." She wrote several novels about the plantation South, especially the Gullah people of the Low Country. She was one of the few white authors who wrote about the African-American experience. Julia Mood was born in Laurens County, South Carolina. Her father was a physician, and she was the third of his four children. Her mother died soon after her birth, and her father later married Janie Brogdon.
------
PARTS
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Context:
${sources that may contain relevant content}

Question: ${the question to be answered}

Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. 

## Testing Chat Adapter Flags with demos

### Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [11]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=True, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Questio

### Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [12]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=True, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=True, multi_turn=False, strict_turn=True

In [13]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=False, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [14]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=False, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=True, strict_turn=True

In [16]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=True, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award'}
{'role': 'assistant', 'content': 'Got 

### Chat Adapter with system_turn=False, multi_turn=True, strict_turn=False

In [18]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=True, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'user', 'content': 'Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=False, strict_turn=True

In [19]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=False, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [20]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=False, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


## Testing Chat Adapter Flags with no demos

### Chat Adapter with system_turn=True, multi_turn=True, strict_turn=True

In [23]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=True, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=True, multi_turn=True, strict_turn=False

In [24]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=True, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=True, multi_turn=False, strict_turn=True

In [25]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=False, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=True, multi_turn=False, strict_turn=False

In [26]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=True, multi_turn=False, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'system', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=True, strict_turn=True

In [27]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=True, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'assistant', 'content': 'Got it.'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=True, strict_turn=False

In [28]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=True, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
[]
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.'}
{'role': 'user', 'content': 'Follow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'}
{'role': 'user', 'content': 'Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=False, strict_turn=True

In [29]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=False, strict_turn=True)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}


### Chat Adapter with system_turn=False, multi_turn=False, strict_turn=False

In [30]:
dsp.settings.configure(lm=turbo, rm=rm)

example = dsp.Example(question=question, demos=demos)
parts = qa_template(example)
prompt = chat_adapter(parts, system_turn=False, multi_turn=False, strict_turn=False)

print("------")
print("parts")
for k, v in parts.items():
    print(k+":")
    print(v)

print("------")
print("prompt")
for k, v in prompt.items():
    print(k+":")
    if k=="messages":
        for turn in v:
            print(turn)
    else:
        print(v)

------
parts
instructions:
Answer questions with short factoid answers.
guidelines:
Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}
rdemos:
['Question: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award']
ademos:
[]
query:
Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer: 
long_query:
False
------
prompt
model:
gpt-3.5-turbo
messages:
{'role': 'user', 'content': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${the question to be answered}\nAnswer: ${a short factoid answer, often between 1 and 5 words}\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?\nAnswer:'}
